In [ ]:
import pandas as pd
import numpy as np

In [ ]:
drinks = pd.read_csv('drinks.csv')

In [26]:
drinks.head()

,Country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


#### Loc/iloc

In [ ]:
# Select columns by name
drinks[['beer_servings','continent']]

In [ ]:
# Select columns with .loc. All rows for Continent
drinks.loc[:,'continent']

In [ ]:
# same but two columns and all rows
drinks.loc[:,['continent','beer_servings']]

In [ ]:
# everything for country Angola
drinks.loc[drinks['country']=='Angola',:]

In [ ]:
# only wine servings in Angola
drinks.loc[drinks['country']=='Angola', ['country','wine_servings']]

In [ ]:
# all countries where wine_servings > 45
drinks['country'].loc[drinks['wine_servings']>145]

In [ ]:
# all countries and continents, and servings where wine_servings > 145
drinks[['country','continent','wine_servings']].loc[drinks['wine_servings']>145]

In [ ]:
#select all even rows
drinks.iloc[::2,:]

In [ ]:
# select all odd columns
drinks.iloc[:,::2]

In [ ]:
# select the last column without using the name
drinks.iloc[:,-1:]

In [ ]:
# select all but the last column without using the name
drinks.iloc[:,:-1]

In [ ]:
# Rename a column in place
# drinks = drinks.rename({'country':'Country'}, axis = 0, inplace = True)

In [ ]:
drinks.rename(columns={"country":"Country"}, inplace = True)

In [ ]:
drinks.columns

In [ ]:
# Selecting everything for a given list of countries
drinks[drinks['Country'].isin(['Algeria','Spain','Canada'])]

In [ ]:
# The opposite of the above: all countries except the given three
drinks[~drinks['Country'].isin(['Algeria','Spain','Canada'])]

In [ ]:
# select columns by data type
drinks.dtypes

In [ ]:
# select first 15 rows
drinks.iloc[:15,:]

In [ ]:
#let's select float64
drinks.select_dtypes(include='float64')

In [ ]:
# let's exclude objects
drinks.select_dtypes(exclude = 'object')

In [ ]:
# For continent: change its type object to type category (saves space)
dtypes ={'continent':'category'}
drinks.astype(dtype = dtypes).dtypes

In [29]:
# Select top3 countries by beer_servings
drinks.nlargest(3,['beer_servings'])

,Country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
117,Namibia,376,3,1,6.8,Africa
45,Czech Republic,361,170,134,11.8,Europe
62,Gabon,347,98,59,8.9,Africa


In [34]:
# Same thing as above but in a more mundane way: pass the index of rows with top3 countries to iloc
drinks.iloc[drinks['beer_servings'].nlargest(3).index,:]

,Country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
117,Namibia,376,3,1,6.8,Africa
45,Czech Republic,361,170,134,11.8,Europe
62,Gabon,347,98,59,8.9,Africa


In [ ]:
# dealing with nans, % of nans in a column, split a string, pivot tables, correlation heatmap, aggregate functions (agg, mean(), group by)
# https://www.youtube.com/watch?v=RlIiVeig3hc
# https://www.youtube.com/watch?v=dPwLlJkSHLo

#### NaNs

In [35]:
# how many missing values are in every column
drinks.isna().sum()

Country                         0
beer_servings                   0
spirit_servings                 0
wine_servings                   0
total_litres_of_pure_alcohol    0
continent                       0
dtype: int64

In [36]:
# % of missing values
drinks.isna().mean()
# alternative
drinks.isna().sum() / len(drinks)

Country                         0.0
beer_servings                   0.0
spirit_servings                 0.0
wine_servings                   0.0
total_litres_of_pure_alcohol    0.0
continent                       0.0
dtype: float64

In [ ]:
# drop columns where more than 10% of values are missing
drinks.dropna(thresh=len(drinks)*0.9, axis = 'columns')

In [ ]:
# fill NaNs with something
drinks.fillna(0) # this case it will be zero
drinks.fillna(method='bfill') # fill with the next value 
drinks.interpolate(method='linear', limit_direction='forward') # interpolate 
drinks.dropna(axis = 1) # drop columns with NaNs

#### Splitting cells

In [39]:
names = pd.DataFrame({'name':['John Arthur Doe', 'Jane Ann Smith'],
                   'location':['Los Angeles, CA', 'Washington, DC']})

In [40]:
names.head()

,name,location
0,John Arthur Doe,"Los Angeles, CA"
1,Jane Ann Smith,"Washington, DC"


In [41]:
# split the column name into three columns and add these columns to the dataset
names[['first','middle','last']]= names['name'].str.split(' ', expand = True)

In [43]:
# split location into two columns but add only one resulting column to the dataset
names['State'] = names['location'].str.split(', ', expand = True)[1]

#### Aggregate functions

In [55]:
# First we need to load a new dataset
chipotle = pd.read_csv('chipotle.csv', error_bad_lines=False, header = None, sep='\\')

In [57]:
chipotle = chipotle.iloc[1:,:]

In [65]:
chipotle.columns.values

array([0], dtype=int64)

In [71]:
chipotle[['order_id','quantity','item_name','choice_description','item_price']] = chipotle.iloc[:,0].str.split('\t', expand = True)

In [73]:
# Drop the first column
chipotle.drop([0],axis = 'columns', inplace = True)

In [74]:
chipotle.head()
# order_id	quantity	item_name	choice_description	item_price

,order_id,quantity,item_name,choice_description,item_price
1,1,1,Chips and Fresh Tomato Salsa,NULL,$2.39
2,1,1,Izze,[Clementine],$3.39
3,1,1,Nantucket Nectar,[Apple],$3.39
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,$2.39
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [77]:
chipotle.dtypes

order_id              object
quantity              object
item_name             object
choice_description    object
item_price            object
dtype: object

In [79]:
# remove the $ sign from item_price in order to convert it to float32 later
chipotle['item_price'] = chipotle['item_price'].str[1:] 

In [90]:
chip_dtypes ={'quantity':'int','item_price': np.float32}
chipotle = chipotle.astype(dtype = chip_dtypes)

In [91]:
chipotle.dtypes

order_id               object
quantity                int32
item_name              object
choice_description     object
item_price            float32
dtype: object

In [93]:
# now we start aggregating
# Calculate the total price of the order: sum item_price for a given order_id
chipotle.groupby('order_id')['item_price'].sum().head()

order_id
1       11.56
10      13.20
100     10.08
1000    20.50
1001    10.08
Name: item_price, dtype: float32

In [101]:
# passing a list of functions to groupby
chipotle.groupby('order_id')['item_price'].agg(['sum','count','std']).head()

,sum,count,std
order_id,,,
1,11.56,4,0.577350
10,13.20,2,3.040559
100,10.08,2,5.586143
1000,20.50,2,1.414214
1001,10.08,2,5.586143


In [102]:
# create a new column showing the total price of each order. Values will repeat b/c the table is on the item level
chipotle['Total Price'] = chipotle.groupby('order_id')['item_price'].transform('sum')

In [103]:
chipotle.head()

,order_id,quantity,item_name,choice_description,item_price,Total Price
1,1,1,Chips and Fresh Tomato Salsa,NULL,2.39,11.56
2,1,1,Izze,[Clementine],3.39,11.56
3,1,1,Nantucket Nectar,[Apple],3.39,11.56
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,2.39,11.56
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,16.98


In [104]:
# calculate the % of item price in the order price: divide one column by another
chipotle['Percentage of order price'] = chipotle['item_price']/chipotle['Total Price']

In [105]:
chipotle.head()

,order_id,quantity,item_name,choice_description,item_price,Total Price,Percentage of order price
1,1,1,Chips and Fresh Tomato Salsa,NULL,2.39,11.56,0.206747
2,1,1,Izze,[Clementine],3.39,11.56,0.293253
3,1,1,Nantucket Nectar,[Apple],3.39,11.56,0.293253
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,2.39,11.56,0.206747
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,16.98,1.000000


#### Pivot tables

In [106]:
# On the Titanic dataset
titanic_train = pd.read_csv('titanic_train.csv')

In [107]:
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [109]:
# straightforward pivot table
titanic_train.pivot_table(index='Sex', columns = 'Pclass', values = 'Survived', aggfunc = 'count', margins = True)

Pclass,1,2,3,All
Sex,,,,
female,94,76,144,314
male,122,108,347,577
All,216,184,491,891


In [112]:
# create bins and add as a column to the dataframe 
titanic_train['Age category'] = pd.cut(titanic_train['Age'], bins = [0,18,25,99], labels = ['child','young adult','adult']).head()

In [113]:
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age category
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,young adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,adult
